In [14]:
from collections import defaultdict, Counter

In [15]:
import pandas as pd
import nltk
# import sklearn as sk
import gensim.models
# from nltk.corpus import stopwords
# import string
import matplotlib.pyplot as plt
# import seaborn as sns
import numpy as np
# from ast import literal_eval
# from predictor import *
import predictor
# sns.set()

clean = pd.read_csv("/Users/aditya/Documents/GitHub/NLP-for-motivation-lab/CleanData(v. analysis).csv")

colname = ['id', 'condition', 'wish', 'outcome', 'outcome2', 'useful', 'mins', 'other', 'reuse', 'clarity', 'benefit', 'interest', 'learn']
clean = clean.iloc[:256,:13]
clean.columns= colname
clean.fillna("", inplace=True)

In [16]:
clean['all'] = clean.wish + " " + clean.outcome + " " + clean.outcome2
clean.drop(columns = ['wish', 'outcome', 'outcome2'], inplace=True)

# clean['all'] = clean['all'].apply(remove_stop)
# model = gensim.models.Word2Vec(clean['all'], min_count = 2)
# model.most_similar('happy')

# clean = pd.get_dummies(clean, columns = ['useful','mins','other','reuse', 'clarity', 'benefit', 'interest', 'learn'])

coders = pd.read_csv('/Users/aditya/Documents/GitHub/NLP-for-motivation-lab/MCII coding replication - coding file(v. analysis).csv')
colname = ['id', 'd1', 'd2', 'd3', 'd4', 'd5', 'd6', 's1', 's2', 's3', 's4', 's5', 's6']
coders = coders.iloc[:256,:13]
coders.columns= colname
coders.fillna(1, inplace=True)
coders['d'] = round(coders.loc[:,'d1':'d6'].mean(axis=1))
coders['s'] = round(coders.loc[:,'s1':'s6'].mean(axis=1))
clean = pd.concat([clean, coders], join='outer', axis=1)
clean.head()
clean.head()

,id,condition,useful,mins,other,reuse,clarity,benefit,interest,learn,...,d5,d6,s1,s2,s3,s4,s5,s6,d,s
0,558af269fdf99b7dfef4d7bd,MC,7.0,1.0,7.0,7.0,7.0,7.0,7.0,7.0,...,2.0,5.0,2.0,5.0,5.0,2.0,5.0,5.0,3.0,4.0
1,55b1add5fdf99b6731f04c5a,MC,1.0,1.0,2.0,1.0,3.0,1.0,2.0,2.0,...,5.0,5.0,4.0,3.0,5.0,4.0,3.0,5.0,5.0,4.0
2,5626a6cfddff3c0011bcd3d8,I,3.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,...,5.0,5.0,3.0,5.0,4.0,3.0,5.0,4.0,4.0,4.0
3,56a5da6181a8ca000d3e7f40,MC,4.0,1.0,4.0,3.0,2.0,3.0,5.0,4.0,...,5.0,5.0,3.0,3.0,4.0,3.0,3.0,4.0,4.0,3.0
4,56df2ea2452e1300056bc7df,MC,3.0,1.0,4.0,4.0,3.0,3.0,4.0,3.0,...,1.0,1.0,2.0,1.0,2.0,2.0,1.0,2.0,1.0,2.0


In [17]:
# for each_col in clean.loc[:,'useful':'learn']:
#     print(each_col)
#     plt.hist(clean[each_col])
#     print(clean[each_col].value_counts())
#     plt.show()

In [18]:
history_scores = defaultdict(dict)

In [19]:
def variation(kind, solver):
    history_scores[f'{solver}'][f'{kind}'] = {}
    for each_col in clean.loc[:,'useful':'learn']:
        mypredictor = predictor.predictor(clean, 'all', kind, solver)
        print("\n" + each_col)
        mypredictor.train_on(each_col)
        history_scores[f'{mypredictor.solver}'][f'{mypredictor.kind}'][f'{each_col}'] = \
        round(mypredictor.test(), 3)

In [20]:
variation('l2', 'liblinear')


useful
{'fit_time': array([0.01347804, 0.01676607, 0.01780295]), 'score_time': array([0.00120687, 0.00108099, 0.00123525]), 'test_score': array([0.62962963, 0.65925926, 0.59701493]), 'train_score': array([0.97769517, 0.94423792, 0.93333333])}
0.7361111111111112

mins
{'fit_time': array([0.01439691, 0.01247191, 0.00864816]), 'score_time': array([0.00142908, 0.00069594, 0.00068879]), 'test_score': array([0.7345679 , 0.80246914, 0.83333333]), 'train_score': array([0.97530864, 0.95987654, 0.92592593])}
0.8953488372093024

other
{'fit_time': array([0.01132703, 0.01012588, 0.01739001]), 'score_time': array([0.000705  , 0.00097609, 0.00105095]), 'test_score': array([0.66115702, 0.57024793, 0.60833333]), 'train_score': array([0.94605809, 0.95020747, 0.9214876 ])}
0.5846153846153846

reuse
{'fit_time': array([0.01792908, 0.0149231 , 0.0114181 ]), 'score_time': array([0.00139189, 0.00109911, 0.00151515]), 'test_score': array([0.63309353, 0.58992806, 0.65217391]), 'train_score': array([0.9458483

In [21]:
variation('l1', 'liblinear')


useful
{'fit_time': array([0.01677513, 0.00918293, 0.00948238]), 'score_time': array([0.00088   , 0.00071216, 0.00076389]), 'test_score': array([0.52592593, 0.4       , 0.44029851]), 'train_score': array([0.53159851, 0.59107807, 0.58518519])}
0.6111111111111112

mins
{'fit_time': array([0.00826287, 0.01328182, 0.01031804]), 'score_time': array([0.00072718, 0.00845718, 0.00072503]), 'test_score': array([0.5617284 , 0.61111111, 0.51234568]), 'train_score': array([0.68518519, 0.66666667, 0.7345679 ])}
0.6627906976744186

other
{'fit_time': array([0.00885296, 0.00905585, 0.00804019]), 'score_time': array([0.00065827, 0.00073028, 0.00112486]), 'test_score': array([0.38016529, 0.3553719 , 0.35833333]), 'train_score': array([0.45643154, 0.42738589, 0.45041322])}
0.35384615384615387

reuse
{'fit_time': array([0.01150298, 0.00892472, 0.00892305]), 'score_time': array([0.00067019, 0.00081038, 0.00081491]), 'test_score': array([0.38129496, 0.32374101, 0.43478261]), 'train_score': array([0.490974

In [9]:
variation('l2', 'lbfgs')


useful
{'fit_time': array([0.62012315, 0.9465332 , 1.37709379]), 'score_time': array([0.00155091, 0.00086093, 0.0009861 ]), 'test_score': array([0.65185185, 0.57777778, 0.64925373]), 'train_score': array([0.97769517, 0.95910781, 0.98518519])}
0.7638888888888888

mins
{'fit_time': array([1.01937985, 0.93932176, 0.95555091]), 'score_time': array([0.00956511, 0.00093317, 0.00062394]), 'test_score': array([0.82716049, 0.82098765, 0.87037037]), 'train_score': array([0.97530864, 0.98148148, 0.97530864])}
0.8023255813953488

other
{'fit_time': array([0.63050866, 0.74788404, 0.63324594]), 'score_time': array([0.00140905, 0.00061917, 0.00106716]), 'test_score': array([0.58677686, 0.62809917, 0.63333333]), 'train_score': array([0.93775934, 0.94605809, 0.94628099])}
0.676923076923077

reuse
{'fit_time': array([0.92675996, 0.44227004, 0.39980578]), 'score_time': array([0.00062585, 0.00092411, 0.00063705]), 'test_score': array([0.62589928, 0.64028777, 0.66666667]), 'train_score': array([0.93862816

In [10]:
variation('l2', 'newton-cg')


useful
{'fit_time': array([0.4216032 , 0.16532707, 0.23309278]), 'score_time': array([0.00081992, 0.00062895, 0.00101304]), 'test_score': array([0.63703704, 0.68888889, 0.6641791 ]), 'train_score': array([0.94052045, 0.94795539, 0.96296296])}
0.7222222222222222

mins
{'fit_time': array([0.08163095, 0.14010024, 0.2360189 ]), 'score_time': array([0.00096798, 0.00085592, 0.00080824]), 'test_score': array([0.79012346, 0.7962963 , 0.82098765]), 'train_score': array([0.97839506, 0.97839506, 0.96296296])}
0.9418604651162791

other
{'fit_time': array([0.16039276, 0.11502695, 0.31518626]), 'score_time': array([0.00093627, 0.00062275, 0.0018158 ]), 'test_score': array([0.56198347, 0.61983471, 0.63333333]), 'train_score': array([0.95850622, 0.95435685, 0.94628099])}
0.676923076923077

reuse
{'fit_time': array([0.19164586, 0.23641896, 0.18518305]), 'score_time': array([0.00125909, 0.0010922 , 0.00102782]), 'test_score': array([0.62589928, 0.56834532, 0.71014493]), 'train_score': array([0.9566787 

In [11]:
history_scores

defaultdict(dict,
            {'liblinear': {'l2': {'useful': 0.681,
               'mins': 0.837,
               'other': 0.723,
               'reuse': 0.635,
               'clarity': 0.731,
               'benefit': 0.686,
               'interest': 0.692,
               'learn': 0.681},
              'l1': {'useful': 0.583,
               'mins': 0.756,
               'other': 0.446,
               'reuse': 0.581,
               'clarity': 0.507,
               'benefit': 0.5,
               'interest': 0.603,
               'learn': 0.406}},
             'lbfgs': {'l2': {'useful': 0.764,
               'mins': 0.802,
               'other': 0.677,
               'reuse': 0.716,
               'clarity': 0.701,
               'benefit': 0.729,
               'interest': 0.744,
               'learn': 0.681}},
             'newton-cg': {'l2': {'useful': 0.722,
               'mins': 0.942,
               'other': 0.677,
               'reuse': 0.703,
               'clarity': 0.627

In [12]:
#checking predictor performance for specificity

mypredictor_coding = predictor.predictor(clean, 'all', k_neighbors = 3)
mypredictor_coding.train_on('s')
mypredictor_coding.test()

{'fit_time': array([1.91910791, 2.43304276, 1.83866692]), 'score_time': array([0.02544808, 0.00064707, 0.00106597]), 'test_score': array([0.83815029, 0.77456647, 0.8372093 ]), 'train_score': array([0.93333333, 0.93043478, 0.94797688])}
0.8695652173913043


0.8695652173913043

In [13]:
mypredictor_coding.predict("Get extra hours at work so I can be able to pay for things my family needs I would be able to take care of my family and what they need or want If I was able to fulfill this goal, I would have extra hours to able to bring more money home to my family and take care of the little things that we need and want. We could be able to go on vacation and do whatever extra stuff that we want.")

TypeError: expected dimension <= 2 array or matrix

In [14]:
pd.DataFrame(mypredictor_coding.model.coef_[0][:-1], 
             mypredictor_coding.vectorizer.get_feature_names(), 
             columns=['coef'])\
            .sort_values(by='coef', ascending=False)

,coef
no,1.043155
from the,0.881520
cloud,0.868776
from,0.656877
best outcome,0.611827
...,...
eating,-0.627745
satisfied,-0.631579
very excited,-0.703780
happy,-0.757581


In [15]:
#checking predictor performance for directness

mypredictor_coding = predictor.predictor(clean, 'all')
mypredictor_coding.train_on('d')
mypredictor_coding.test()

{'fit_time': array([0.40769386, 0.39222288, 0.58424807]), 'score_time': array([0.00094795, 0.00061798, 0.00176406]), 'test_score': array([0.79054054, 0.76190476, 0.80272109]), 'train_score': array([0.93197279, 0.93559322, 0.91864407])}
0.7692307692307693


0.7692307692307693

In [16]:
mypredictor_coding.predict("Get extra hours at work so I can be able to pay for things my family needs I would be able to take care of my family and what they need or want If I was able to fulfill this goal, I would have extra hours to able to bring more money home to my family and take care of the little things that we need and want. We could be able to go on vacation and do whatever extra stuff that we want.")

ValueError: blocks[0,:] has incompatible row dimensions. Got blocks[0,1].shape[0] == 2, expected 1.

In [17]:
pd.DataFrame(mypredictor_coding.model.coef_[0][:-1], 
             mypredictor_coding.vectorizer.get_feature_names(), 
             columns=['coef'])\
            .sort_values(by='coef', ascending=False)

,coef
nice,0.742462
no,0.722316
and,0.676004
clean,0.611332
done for,0.609716
...,...
happy,-0.433513
money,-0.433581
pizza,-0.457694
great,-0.499986


In [18]:
#effect of imbalance? 
#interpreting coeff? 
#useful exercise? What can be done? 
#how better approximate coders? 

In [19]:
coder1_specific_d = clean[['all','d1','d2','d3']][:128]
coder1_specific_d['d'] = round(coder1_specific_d[coder1_specific_d.columns[-3:]].mean(axis=1))
coder2_specific_d = clean[['all','d1','d2','d3']][128:]
coder2_specific_d['d'] = round(coder2_specific_d[coder2_specific_d.columns[-3:]].mean(axis=1))
coder3_specific_d = clean[['all','d4','d5','d6']][:128]
coder3_specific_d['d'] = round(coder3_specific_d[coder3_specific_d.columns[-3:]].mean(axis=1))
coder4_specific_d = clean[['all','d4','d5','d6']][128:]
coder4_specific_d['d'] = round(coder4_specific_d[coder4_specific_d.columns[-3:]].mean(axis=1))

In [20]:
coder1_specific_d_predictor = predictor.predictor(coder1_specific_d, 'all')
coder1_specific_d_predictor.train_on('d')
coder1_specific_d_predictor.test()

{'fit_time': array([0.20036626, 0.17820287, 0.18497896]), 'score_time': array([0.00254297, 0.00054288, 0.00111198]), 'test_score': array([0.81538462, 0.75384615, 0.78461538]), 'train_score': array([0.96153846, 0.96923077, 0.96923077])}
0.7428571428571429


0.7428571428571429

In [28]:
coder2_specific_d_predictor = predictor.predictor(coder2_specific_d, 'all')
coder2_specific_d_predictor.train_on('d')
coder2_specific_d_predictor.test()

{'fit_time': array([0.08938217, 0.09994078, 0.08066916]), 'score_time': array([0.00052595, 0.00051808, 0.0007019 ]), 'test_score': array([0.55102041, 0.71428571, 0.64583333]), 'train_score': array([0.91752577, 0.93814433, 0.91836735])}
0.6153846153846154


0.6153846153846154

In [22]:
coder3_specific_d_predictor = predictor.predictor(coder3_specific_d, 'all')
coder3_specific_d_predictor.train_on('d')
coder3_specific_d_predictor.test()

{'fit_time': array([0.21390104, 0.17577696, 0.24685884]), 'score_time': array([0.00055027, 0.00065899, 0.00054193]), 'test_score': array([0.8       , 0.76923077, 0.75384615]), 'train_score': array([0.96923077, 0.96153846, 0.96923077])}
0.8285714285714286


0.8285714285714286

In [23]:
coder4_specific_d_predictor = predictor.predictor(coder4_specific_d, 'all', k_neighbors= 1)
coder4_specific_d_predictor.train_on('d')
coder4_specific_d_predictor.test()

ValueError: Expected n_neighbors <= n_samples,  but n_samples = 1, n_neighbors = 2

In [27]:
Counter(coder4_specific_d.d)

Counter({3.0: 45, 4.0: 15, 2.0: 53, 1.0: 14, 5.0: 1})